## Масштабирование системы


## Балансировка нагрузки (Load Balancing)
Балансировщик — это регулировщик, который распределяет трафик между серверами.

**Алгоритмы:**
1.  **Round-robin**: По кругу (1 -> 2 -> 3 -> 1).
2.  **Least connections**: Тому, у кого меньше всего активных соединений.
3.  **Least response time**: Самый быстрый сервер.
4.  **IP Hash**: Чтобы один и тот же юзер всегда попадал на один и тот же сервер (Sticky session).

---

## Распределение данных (Partitioning / Sharding)
Когда данные не влезают на один сервер, мы их режем на части.

### Способы разбиения:
1.  **По диапазону (Range Based)**: ID 1-1000 -> Сервер A, 1001-2000 -> Сервер B.
    *   *Проблема*: Неравномерность (все новые юзеры пишут в последний шард).
2.  **По хешу (Hash Based)**: `Shard_ID = Hash(Key) % N_Servers`.
    *   *Плюс*: Равномерное распределение.
    *   *Минус*: При добавлении сервера меняется N, нужно перегонять почти все данные (Rebalancing).
3.  **Консистентное хеширование (Consistent Hashing)**:
    *   Представим круг (кольцо) от 0 до $2^{32}$.
    *   Сервера — точки на круге.
    *   Ключ — точка на круге.
    *   Данные храним на ближайшем сервере по часовой стрелке.
    *   *Плюс*: При добавлении сервера перемещаются только ключи ближайшего соседа.

---

## Репликация и Отказоустойчивость
Как не потерять данные, если сервер сгорит? Хранить копии!

### Стратегии репликации:
1.  **Leader-Follower (Master-Slave)**:
    *   **Leader**: Принимает все **Записи** (Writes) и Чтения.
    *   **Followers**: Копируют данные с Лидера. Принимают только **Чтения** (Reads).
    *   *Если Лидер упал*: Выбираем нового из Фоловеров.
2.  **Master-Master**: Пишем в любой сервер. Сложно разрешать конфликты.

### Кворум (Quorum)
В системах без единого лидера (например, Cassandra).
*   **N**: Число реплик (например, 3).
*   **W**: Число подтверждений записи (Write Quorum).
*   **R**: Число подтверждений чтения (Read Quorum).
*   **Условие консистентности**: `W + R > N`.
    *   Если пишем на 2 из 3, и читаем с 2 из 3, то гарантированно пересечемся и увидим свежую запись.


## Практика: Консистентное хеширование (Consistent Hashing)


### Зачем это нужно?
Обычный подход `hash(key) % N` плох тем, что при изменении `N` (добавили сервер) почти все ключи меняют свое местоположение.
Консистентное хеширование решает эту проблему, минимизируя перемещение данных.

Ниже простой пример реализации кольца хеширования на Python.


In [ ]:

import hashlib

class ConsistentHash:
    def __init__(self, nodes=None, replicas=3):
        self.replicas = replicas  # Количество виртуальных нод на один сервер (для равномерности)
        self.ring = {}            # Кольцо: Hash -> Node Name
        self.sorted_keys = []     # Отсортированные хеши кольца
        
        if nodes:
            for node in nodes:
                self.add_node(node)

    def _hash(self, key):
        # Используем MD5 для хеширования
        return int(hashlib.md5(key.encode('utf-8')).hexdigest(), 16)

    def add_node(self, node):
        # Добавляем сервер + его виртуальные реплики на кольцо
        for i in range(self.replicas):
            key = self._hash(f"{node}:{i}")
            self.ring[key] = node
            self.sorted_keys.append(key)
        self.sorted_keys.sort()
        print(f"✅ Добавлен сервер: {node}")

    def remove_node(self, node):
        # Удаляем сервер и его реплики
        for i in range(self.replicas):
            key = self._hash(f"{node}:{i}")
            del self.ring[key]
            self.sorted_keys.remove(key)
        print(f"❌ Удален сервер: {node}")

    def get_node(self, key):
        if not self.ring:
            return None
        
        hash_val = self._hash(key)
        
        # Ищем первый узел на кольце, чей хеш >= хешу ключа (движение по часовой стрелке)
        for node_hash in self.sorted_keys:
            if hash_val <= node_hash:
                return self.ring[node_hash]
        
        # Если не нашли (мы в конце кольца), берем первый узел (замыкаем круг)
        return self.ring[self.sorted_keys[0]]

# --- Тестирование ---
ch = ConsistentHash(nodes=['Server-A', 'Server-B', 'Server-C'])

keys = ['user_1', 'user_2', 'user_3', 'order_123', 'payment_555']

print("\n--- Распределение ключей ---")
for k in keys:
    print(f"Ключ '{k}' -> {ch.get_node(k)}")

# Симуляция падения сервера B
print("\n--- Падение Server-B ---")
ch.remove_node('Server-B')

print("\n--- Перераспределение (Заметьте: user_1 и user_3 остались на месте!) ---")
for k in keys:
    print(f"Ключ '{k}' -> {ch.get_node(k)}")
